## Типовой расчёт
Провести статистическое моделирование отказов изделий, у которых время наработки на отказ распределены по экспоненциальному, нормальному и логнормальному закону. Смоделировать выборки объёмом по 50000 изделий для различных значений параметров распределений. По этим данным определить средние наработки на отказ, построить гистограммы распределения времени наступления отказов, зависимости вероятности безотказной работы от времени, частоты отказов и интенсивности отказов от времени. Разработать пользовательский интерфейс для приложения.

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from ipywidgets import widgets


def gen_expo(lambd, n=50000):
    return np.random.exponential(lambd, n)

def gen_normal(mu, s, n=50000):
    return np.abs(np.random.normal(mu, s, n))

def gen_lognormal(mean, sigma, n=50000):
    return np.random.lognormal(mean, sigma, n)



def calc(t, arr, dt):
    arr = np.array(sorted(arr))
    N = len(arr)
    n1 = np.zeros(len(t))
    alpha_t = np.zeros(len(t))
    lambd_t = np.zeros(len(t))
    for i in range(len(t)):
        No = np.sum(np.where(arr < t[i], 1, 0))
        n1[i] = No
        n_dt = np.sum(np.where((arr > (t[i] - dt)/2) & (arr < (t[i] + dt)/2), 1, 0))
        alpha_t[i] = n_dt / (N*dt)
        Ni1 = np.sum(np.where(arr > (t[i]-dt)/2, 1, 0))
        Ni2 = np.sum(np.where(arr > (t[i]+dt)/2, 1, 0))
        lambd_t[i] = n_dt / (((Ni1+Ni2) / 2)*dt)
    p_t = (N - np.array(n1)) / N
    Tmn = np.trapz(p_t)
    return p_t, lambd_t, alpha_t, Tmn


def draw(data, t, p_t, lambd_t, alpha, Tmn):
    fig = plt.figure(figsize=(7, 10))
    ax1 = fig.add_subplot(411)
    ax2 = fig.add_subplot(412)
    ax3 = fig.add_subplot(413)
    ax4 = fig.add_subplot(414)
    ax1.set_title('Зависимость вероятности безотказной работы от времени')
    ax1.set_xlabel('t')
    ax1.set_ylabel('P(t)')
    ax2.set_title('Зависимость интенсивности отказов от времени')
    ax2.set_xlabel('t')
    ax2.set_ylabel('lambda(t)')
    ax3.set_title('Зависимость частоты отказов от времени')
    ax3.set_xlabel('t')
    ax3.set_ylabel('alpha(t)')
    ax4.set_title('Гистограмма распределения времени наступления отказов')
    ax1.plot(t, p_t)
    ax2.plot(t, lambd_t)
    ax3.plot(t, alpha)
    ax4.hist(data)
    plt.tight_layout()
    print('Средняя наработка на отказ: ', Tmn)
    plt.show()


def main(*args):
    dt = w_dt.value
    if dist.value == 'expo':
        lambd = w_exp_lambda.value
        data = gen_expo(lambd)
    elif dist.value == 'normal':
        mu = w_normal_mu.value
        s = w_normal_s.value
        data = gen_normal(mu, s)
    elif dist.value == 'lognormal':
        mean = w_lognormal_mean.value
        sigma = w_lognormal_sigma.value
        data = gen_lognormal(mean, sigma)
    with out2:
        clear_output(wait=True)
        t = np.linspace(1, 10000, 1000)
        draw(data, t, *calc(t, data, dt))

    

out1 = widgets.Output(layout={'width': '50%'})
out2 = widgets.Output(layout={'width': '50%'})
title = widgets.HTML('<h4>Статистическое моделирование отказов изделий</h4>')
dist = widgets.RadioButtons(options={'Экспоненциальное': 'expo',
                                    'Нормальное': 'normal',
                                    'Логнормальное': 'lognormal',})
w_dt_L = widgets.Label('dt')
w_dt = widgets.FloatSlider(description='', min=10, max=1000, step=1, value=400)
params_L = widgets.Label('Параметры распределений')
w_exp_L = widgets.Label('Экспоненциальное распределение')
w_exp_lambda = widgets.IntSlider(description='lambda', min=1000, max=100000, step=100, value=10000)
w_normal_L = widgets.Label('Нормальное распределение')
w_normal_mu = widgets.FloatSlider(description='mu', min=1000, max=100000, step=100, value=10000)
w_normal_s = widgets.FloatSlider(description='s', min=1000, max=100000, step=100, value=10000)
w_lognormal_L = widgets.Label('Логнормальное распределение')
w_lognormal_mean = widgets.FloatSlider(description='mean', min=1, max=100, step=1, value=10)
w_lognormal_sigma = widgets.FloatSlider(description='sigma', min=1, max=100, step=1, value=10)
button = widgets.Button(description='Запустить', button_style='primary')
button.on_click(main)
display(widgets.HBox([out1, out2]))
with out1:
    display(title,
            dist,
            w_dt_L,
            w_dt,
            params_L,
            w_exp_L,
            w_exp_lambda,
            w_normal_L,
            w_normal_mu, 
            w_normal_s,
            w_lognormal_L,
            w_lognormal_mean,
            w_lognormal_sigma,
            button,
            )